In [1]:
import pandas as pd
from decimal import Decimal
import json
import numpy as np
from pprint import pprint
from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v2.0-beta3 (12/Dec/2022)


# Carbon Simulation - Demo-5-7 Simulator Import_Export NEW

Here we demonstrate the preferred setup to import and export json test files from the simulator using the new format

In [2]:
def load_json(filename):     
    '''
    :filename:  the convention is to specify the trade by source or trade by target in the filename i.e. tradeByTargetAmount.json
                this is necessary as the `is_by_target` field specifies the trade direction for the actionList
    '''   
    f = open(filename , "r")
    bm = f.read()
    loaded_json = json.loads(bm)
    f.close()
    return(loaded_json)

def decimalize_dict(dicta):
    deci_dict = {}
    for k,v in dicta.items():
        if k != 'token':
            v = Decimal(v)
        deci_dict[k] = v
    return(deci_dict)

def calc_y_int(liquidity, highestRate, lowestRate, marginalRate):
    return(liquidity * (highestRate.sqrt() - lowestRate.sqrt()) / (marginalRate.sqrt() - lowestRate.sqrt()))

def parse_order(order0, order_count):
    order0 = decimalize_dict(order0)
    order0['order'] = order_count
    order0_df = pd.DataFrame.from_dict(order0, 'index', columns=[f'{order_count}'])
    return(order0_df)

def parse_strategy(strategy, strat_count, order_count):
    orders = pd.DataFrame()
    orderList = strategy['orders']
    for order in orderList:
        order_df = parse_order(order, order_count)
        orders = pd.concat([orders,order_df.T])
        order_count += 1
    orders['strategy'] = strat_count
    strat_count += 1
    return(orders, strat_count, order_count)

def format_json_as_df(input_json, scenario_index = 0):
    '''
    :json:             the loaded json file
    :scenario_index:   each json file has multiple scenarios, make a selection
    '''
    actionList = input_json[scenario_index]['tradeActions']
    is_by_target = not input_json[scenario_index]['tradeBySource']
    sourceToken = input_json[scenario_index]['sourceToken']
    targetToken = input_json[scenario_index]['targetToken']
    
    print(f'The number of scenarios is {len(input_json)}')
    print(f'You selected scenario: {scenario_index}')  
    print(f'The number of actions in this scenario are: {len(actionList)}')

    if scenario_index >= len(input_json):
        print('Please selected a valid scenario_index')
        return()
    else:
        strats = pd.DataFrame()
        strat_count = 0
        order_count = 0
        for strategy in input_json[scenario_index]['strategies']:
            orders, strat_count, order_count = parse_strategy(strategy, strat_count, order_count)
            strats = pd.concat([strats, orders])
        return(strats, actionList, is_by_target, sourceToken, targetToken)

def parse_actions(Sim, actionList, strats, is_by_target, sourceToken, targetToken):
    '''
    parse_actions processes trades from the actionList

    :Sim:           the specific simulator environment to trade against
    :actionList:    the list of actions provides from the test json
    :strats:        the preprocessed dataframe of strategies from the test json
    :is_by_target:  the specification of trading by target or by source
    '''
    for action in actionList:
        strategyId = int(action['strategyId']) - 1
        targetStrategy = strats.query(f"strategy=={strategyId}")
        tokenAmount = Decimal(action['amount'])
        if not is_by_target:
            targetOrderId = targetStrategy.query(f"token=='{sourceToken}'")['order'].values[0]
            Sim.amm_buys(targetToken, tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_source
            print(sourceToken, targetOrderId, tokenAmount)
        else:
            targetOrderId = targetStrategy.query(f"token=='{targetToken}'")['order'].values[0]
            Sim.amm_sells(targetToken, tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_target
            print(targetToken, targetOrderId, tokenAmount)
        
def compare_results(strats, initial_orders, final_orders):
    '''
    :strats:            the preprocessed dataframe of strategies from the test json
    :initial_orders:    the state of the simulator orders immediately after importing
    :final_orders:      the state of the simulator orders immediately after performing trade actions
    '''

    df = pd.concat([strats[['newLiquidity', 'newMarginalRate']].reset_index(drop=True), final_orders[['tkn', 'y', 'p_marg']].reset_index(drop=True)], axis=1)
    df.columns = ['json_liquidity', 'json_price', 'tkn', 'sim_liquidity', 'sim_price']
    df = pd.concat([df.reset_index(drop=True), strats[['delta_liquidity']].reset_index(drop=True), (final_orders['y'] - initial_orders['y']).reset_index(drop=True)], axis=1)

    df = df[['tkn','json_price', 'sim_price', 'json_liquidity', 'sim_liquidity', 'delta_liquidity', 'y']]
    # Remember that the prices for one side of the strategy are flipped when imported to the simulator so they need to be inverted for comparison here
    df.loc[:,'sim_price'] = [1/df.sim_price[i] if i%2==1 else df.sim_price[i] for i in df.index]
    df.columns = ['tkn','json_price', 'sim_price', 'json_liquidity', 'sim_liquidity', 'delta_json_liquidity', 'delta_sim_liquidity']
    return(df)

def check_results(results, tolerance, short_long = 'short'):
    '''
    :results:       a df output from compare_results
    :tolerance:     the absolute tolerance acceptable between compared values, default 1 wei
    :short_long:    "short" returns everything in the table passed
                    "long" returns itemized list of what passed
    '''
    test_marg_rates = np.isclose(results.json_price.astype(float), results.sim_price.astype(float), rtol=tolerance)
    test_final_liq = np.isclose(results.json_liquidity.astype(float), results.sim_liquidity.astype(float), rtol=tolerance)
    test_delta_liq = np.isclose(results.delta_json_liquidity.astype(float), results.delta_sim_liquidity.astype(float), rtol=tolerance)

    if short_long == 'short':
        print('Short check')
        print('Testing marginal rate:', test_marg_rates.all())
        assert test_marg_rates.all()
        print('Testing final liquidity:', test_final_liq.all())
        assert test_final_liq.all()
        print('Testing delta liquidity:', test_delta_liq.all())
        assert test_delta_liq.all()
    
    else:
        return(pd.DataFrame(zip(test_marg_rates, test_final_liq, test_delta_liq), columns=['newMarginalRate', 'newLiquididy', 'deltaLiquididty'], index=results.index))


def test_all_scenarios(filename, tolerance):

    '''
    :tolerance: the absolute acceptable tolerance
    '''

    # load the json
    bm_json, is_by_target = load_json(filename)

    # loop over each scenario
    for scenario_index in range(len(bm_json)):
        print('\n')
        # read in strategies and actions
        strats, actionList = format_json_as_df(bm_json, scenario_index = scenario_index)

        # initialize simulator
        FastSim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='fast', raiseonerror=True)

        # add strategies to simulator
        for i in strats.order[::2]:
            FastSim.add_strategy(
                tkn='USDC',
                amt_sell=strats.liquidity[i],
                psell_start=strats.highestRate[i],
                psell_end=strats.lowestRate[i],
                amt_buy=strats.liquidity[i+1],
                pbuy_start=1/strats.highestRate[i+1],
                pbuy_end=1/strats.lowestRate[i+1],
                psell_marginal=strats.marginalRate[i],
                pbuy_marginal=1/strats.marginalRate[i+1],
            )
        # snapshot the orders on the simulator
        initial_orders = FastSim.state()['orders']

        # process the trades from the action list
        parse_actions(FastSim, actionList, strats, is_by_target)

        # snapshot the orders on the simulator post-trade
        final_orders = FastSim.state()['orders']
        
        # generate a comparison report
        results = compare_results(strats, initial_orders, final_orders)

        # check that the difference between simulator and json in within 1 wei
        check_results(results, tolerance = tolerance, short_long = 'short')
    
    print('All tests complete')

def generate_json_actions_new(trade_a):

    json_actions = {}
    sub_actions_list = []

    if trade_a['success'] & trade_a['trades'].exec.all():
        subdf = trade_a['trades'].query('subid!="A"').copy()
        subdf.reset_index(inplace=True, drop=True)
        trade_is_by_target = trade_a['is_by_target']
        tkn = trade_a['tkn']
        if trade_is_by_target:
            sourceToken = FastSim.carbon_pair.other(tkn)
            targetToken = tkn
        else:
            sourceToken = tkn
            targetToken = FastSim.carbon_pair.other(tkn)

        print(f'trade_is_by_target: {trade_is_by_target}')

        for i in subdf.index:
            sub_actions = {}
            sub_actions['strategyId'] = (subdf.routeix[i]//2) + 1 # plus one to encode for contracts
            if trade_is_by_target:
                sub_actions['amount'] = subdf.amt1[i]
            else:
                sub_actions['amount'] = subdf.amt2[i]
            sub_actions_list += [sub_actions]
            
    json_actions['tradeActions'] = sub_actions_list
    return(json_actions, trade_is_by_target, sourceToken, targetToken)

def generate_json_strategies_new(initial_orders):
    json_strats = {}
    json_orders = {}
    json_orders_list = []
    sub_dict_list = []
    for i in initial_orders.index[::2]:
        json_orders = {}
        sub_dict0 = {}
        sub_dict0['token'] = str(initial_orders.tkn[i+1])
        sub_dict0['liquidity'] = str(initial_orders.y[i+1])
        sub_dict0['lowestRate'] = str(initial_orders.p_end[i+1] - 0.000000001)
        sub_dict0['highestRate'] = str(initial_orders.p_start[i+1] + 0.000000001)
        sub_dict0['marginalRate'] = str(initial_orders.p_marg[i+1])
        
        sub_dict1 = {}
        sub_dict1['token'] = str(initial_orders.tkn[i])
        sub_dict1['liquidity'] = str(initial_orders.y[i])
        sub_dict1['lowestRate'] = str(1/(initial_orders.p_end[i] + 0.000000001))
        sub_dict1['highestRate'] = str(1/(initial_orders.p_start[i] - 0.000000001))
        sub_dict1['marginalRate'] = str(1/initial_orders.p_marg[i])
        
        json_orders['orders'] = [sub_dict0,sub_dict1]
        json_orders_list += [json_orders]
    json_strats['strategies'] = json_orders_list
    return(json_strats)

def generate_json_expectedResults_new(final_orders):
    expectedResults = {}
    sub_dict_list = []
    for i in final_orders.index[::2]:
        sub_dict0 = {}
        sub_dict0['newLiquidity'] = str(final_orders.y[i+1])
        sub_dict0['newMarginalRate'] = str(final_orders.p_marg[i+1])
        
        sub_dict1 = {}
        sub_dict1['newLiquidity'] = str(final_orders.y[i])
        sub_dict1['newMarginalRate'] = str(1/final_orders.p_marg[i])
        sub_dict_list += [sub_dict0,sub_dict1]
    expectedResults['expectedResults'] = sub_dict_list
    return(expectedResults)

def format_simulator_json_new(json_strats, json_actions, expectedResults, trade_is_by_target, sourceToken, targetToken):
    json_simulator = []
    simulator_setup = {}
    simulator_setup['tradeBySource'] = False if trade_is_by_target else True
    simulator_setup['sourceToken'] = sourceToken
    simulator_setup['targetToken'] = targetToken
    simulator_setup.update({**json_strats, **json_actions, **expectedResults})
    json_simulator += [simulator_setup]
    return(json_simulator)

# Structure

In [3]:
testData = [
    {
        "tradeBySource": False,
        "sourceToken": 'ETH',
        "targetToken": 'USDC',
        "strategies": [
            {   
                "orders":[
                    {
                        "token": "ETH" ,
                        "liquidity": '864192',
                        "lowestRate": '8.0000',
                        "highestRate": '9.0000',
                        "marginalRate": '9.0000'
                    },
                    {
                        "token": "USDC" ,
                        "liquidity": '2962962',
                        "lowestRate": '1.0000',
                        "highestRate": '2.0000',
                        "marginalRate": '2.0000'
                    }
                ]
            }  
        ],
        "tradeActions": [
            {
                "strategyId": 1,
                "amount": '1000'
            }
        ],
        "expectedResults": [
            {
                "liquidity": '866192',
                "marginalRate": '9.0000'
            },
            {
                "liquidity": '2958965',
                "marginalRate": '1.9984'
            }
        ]
    }
]

# Export

## Initialize a simulator and add some strategies

In [40]:
# New Sim
FastSim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='fast', raiseonerror=True)

In [41]:
# Add strategies
FastSim.add_strategy("ETH", 10, 2000, 2500, 10000, 1040, 1020, "ETH/USDC", 2100, 1030)
FastSim.add_strategy("ETH", 10, 2010, 2810, 10000, 1510, 1210, "ETH/USDC", 2100, 1250)
FastSim.add_strategy("ETH", 10, 2020, 2220, 10000, 1060, 1050, "ETH/USDC", 2200, 1055)
FastSim.add_strategy("ETH", 10, 2030, 2060, 10000, 1750, 1055, "ETH/USDC", 2035, 1700)
FastSim.add_strategy("ETH", 10, 2040, 2080, 10000, 1250, 1040, "ETH/USDC", 2050, 1050)

# Snapshot the simulator orders
initial_orders = FastSim.state()['orders']

## Do a trade and process it into an actionList

In [42]:
# Do a trade
# trade_a = FastSim.amm_sells('ETH', 29, execute=True) # route_trade_by_target
# trade_a = FastSim.amm_buys('ETH', 11, execute=True) # route_trade_by_source
# trade_a = FastSim.amm_sells('USDC', 31000, execute=True) # route_trade_by_target
trade_a = FastSim.amm_buys('USDC', 42000, execute=True) # route_trade_by_source

# Generate the actions component of the json
json_actions, trade_is_by_target, sourceToken, targetToken  = generate_json_actions_new(trade_a)

trade_is_by_target: False


In [43]:
# Snapshot the orders
final_orders = FastSim.state()['orders']
final_orders

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,12.95803,10.00000,ETH,False,2000.00000,2500.00000,2100.00000,USDC per ETH,1
1,1,ETHUSDC,USDC,19951.57300,10000.00000,USDC,False,1040.00000,1020.00000,1030.00000,USDC per ETH,0
2,2,ETHUSDC,ETH,11.63396,10.00000,ETH,False,2010.00000,2810.00000,2100.00000,USDC per ETH,3
3,3,ETHUSDC,USDC,71432.10220,10000.00000,USDC,False,1510.00000,1210.00000,1250.00000,USDC per ETH,2
4,4,ETHUSDC,ETH,106.58194,9.60200,ETH,False,2020.00000,2220.00000,2200.79082,USDC per ETH,5
5,5,ETHUSDC,USDC,19976.33126,10875.92637,USDC,False,1060.00000,1050.00000,1055.43853,USDC per ETH,4
6,6,ETHUSDC,ETH,12.02214,0.00000,ETH,False,2030.00000,2060.00000,2060.00000,USDC per ETH,7
7,7,ETHUSDC,USDC,30474.61843,30474.61843,USDC,False,1750.00000,1055.00000,1750.00000,USDC per ETH,6
8,8,ETHUSDC,ETH,13.38225,0.00000,ETH,False,2040.00000,2080.00000,2080.00000,USDC per ETH,9
9,9,ETHUSDC,USDC,200831.31063,30649.45520,USDC,False,1250.00000,1040.00000,1070.80087,USDC per ETH,8


## Generate the strategies component of the json

In [44]:
# Generate the strategies component of the json
json_strats = generate_json_strategies_new(initial_orders)
expectedResults = generate_json_expectedResults_new(final_orders)

## Combine and format the jsons

In [45]:
json_simulator = format_simulator_json_new(json_strats, json_actions, expectedResults, trade_is_by_target, sourceToken, targetToken)
pprint(json_simulator, sort_dicts=False)

[{'tradeBySource': True,
  'sourceToken': 'ETH',
  'targetToken': 'USDC',
  'strategies': [{'orders': [{'token': 'USDC',
                              'liquidity': '10000.0',
                              'lowestRate': '1019.999999999',
                              'highestRate': '1040.000000001',
                              'marginalRate': '1030.0'},
                             {'token': 'ETH',
                              'liquidity': '10.0',
                              'lowestRate': '0.00039999999999984',
                              'highestRate': '0.00050000000000025',
                              'marginalRate': '0.0004761904761904762'}]},
                 {'orders': [{'token': 'USDC',
                              'liquidity': '10000.0',
                              'lowestRate': '1209.999999999',
                              'highestRate': '1510.000000001',
                              'marginalRate': '1250.0'},
                             {'token': 'ETH',
        

## Optionally Save

In [46]:
from datetime import datetime
str_date = datetime.today().strftime("%Y%m%d")
unique_id = datetime.now().strftime("%f")
    
export_filename = f"{str_date}_test_{unique_id}.json"

out_file = open(export_filename, "w")
json.dump(json_simulator, out_file, indent=4)
out_file.close()

print(f'Saved as {export_filename}')

Saved as 20221221_test_480667.json


# Simply run the whole check on every scenario
Using test_all_scenarios or follow the notebook for step-by-step processing

In [47]:
filename='20221221_test_480667.json'
# test_all_scenarios(filename, tolerance= 0.1)

# Import

First we load the json test file and return a df of the strategies and a list of actions

** If you want to use a specific scenario you need to specify the scenario_index here **

In [48]:
# Specify the scenario_index for a different scenario
input_json = load_json(filename)
strats, actionList, is_by_target, sourceToken, targetToken = format_json_as_df(input_json, scenario_index = 0)
strats

The number of scenarios is 1
You selected scenario: 0
The number of actions in this scenario are: 3


,token,liquidity,lowestRate,highestRate,marginalRate,order,strategy
0,USDC,10000.0,1019.999999999,1040.000000001,1030.0,0,0
1,ETH,10.0,0.00039999999999984,0.00050000000000025,0.0004761904761904762,1,0
2,USDC,10000.0,1209.999999999,1510.000000001,1250.0,2,1
3,ETH,10.0,0.0003558718861208698,0.0004975124378111928,0.0004761904761904762,3,1
4,USDC,10000.0,1049.999999999,1060.000000001,1055.0,4,2
5,ETH,10.0,0.00045045045045024755,0.0004950495049507401,0.00045454545454545465,5,2
6,USDC,10000.0,1054.999999999,1750.000000001,1700.0000000000002,6,3
7,ETH,10.0,0.0004854368932036478,0.0004926108374386663,0.0004914004914004914,7,3
8,USDC,10000.0,1039.999999999,1250.000000001,1049.9999999999998,8,4
9,ETH,10.0,0.00048076923076899964,0.0004901960784316128,0.0004878048780487804,9,4


# Initialize the Simulator and create the orders
Here the orders are entered into the simulator and the prices for one side of the strategy are flipped to coincide with the price convention

In [49]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

FastSim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='fast', raiseonerror=True)

for i in strats.order[::2]:
    FastSim.add_strategy(
        tkn='USDC',
        amt_sell=Decimal(str(strats.liquidity[i])),
        psell_start=strats.highestRate[i],
        psell_end=strats.lowestRate[i],
        amt_buy=strats.liquidity[i+1],
        pbuy_start=1/strats.highestRate[i+1],
        pbuy_end=1/strats.lowestRate[i+1],
        psell_marginal=strats.marginalRate[i],
        pbuy_marginal=1/strats.marginalRate[i+1],
    )
initial_orders = FastSim.state()['orders']
initial_orders

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,19951.57300,10000.00000,USDC,False,1040.00000,1020.00000,1030.00000,USDC per ETH,1
1,1,ETHUSDC,ETH,12.95803,10.00000,ETH,False,2000.00000,2500.00000,2100.00000,USDC per ETH,0
2,2,ETHUSDC,USDC,71432.10220,10000.00000,USDC,False,1510.00000,1210.00000,1250.00000,USDC per ETH,3
3,3,ETHUSDC,ETH,11.63396,10.00000,ETH,False,2010.00000,2810.00000,2100.00000,USDC per ETH,2
4,4,ETHUSDC,USDC,19976.33126,10000.00000,USDC,False,1060.00000,1050.00000,1055.00000,USDC per ETH,5
5,5,ETHUSDC,ETH,106.58194,10.00000,ETH,False,2020.00000,2220.00000,2200.00000,USDC per ETH,4
6,6,ETHUSDC,USDC,10687.91421,10000.00000,USDC,False,1750.00000,1055.00000,1700.00000,USDC per ETH,7
7,7,ETHUSDC,ETH,12.02214,10.00000,ETH,False,2030.00000,2060.00000,2035.00000,USDC per ETH,6
8,8,ETHUSDC,USDC,200831.31061,10000.00000,USDC,False,1250.00000,1040.00000,1050.00000,USDC per ETH,9
9,9,ETHUSDC,ETH,13.38225,10.00000,ETH,False,2040.00000,2080.00000,2050.00000,USDC per ETH,8


# Parse the actions to perform the trades

In [50]:
parse_actions(FastSim, actionList, strats, is_by_target, sourceToken, targetToken)

FastSim.state()['trades']

ETH 7 20474.61843399999997927807271480560302734375
ETH 9 20649.45519899999999324791133403778076171875
ETH 5 875.9263680000000249492586590349674224853515625


,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #7,False,True,None,9.99906,ETH,20472.69131,USDC,ETHUSDC,7,1,2047.46067,USDC per ETH
0,0,0,A,AMM sells 10ETH buys 20473USDC,True,True,None,9.99906,ETH,20472.69131,USDC,ETHUSDC,[7],1,2047.46067,USDC per ETH
0,1.0,1,0,route #9,False,True,None,9.99960,ETH,20648.62069,USDC,ETHUSDC,9,1,2064.94492,USDC per ETH
0,1,1,A,AMM sells 10ETH buys 20649USDC,True,True,None,9.99960,ETH,20648.62069,USDC,ETHUSDC,[9],1,2064.94492,USDC per ETH
0,2.0,2,0,route #5,False,True,None,0.39800,ETH,875.75127,USDC,ETHUSDC,5,1,2200.39538,USDC per ETH
0,2,2,A,AMM sells 0ETH buys 876USDC,True,True,None,0.39800,ETH,875.75127,USDC,ETHUSDC,[5],1,2200.39538,USDC per ETH


# Display the status of the orders post-trade

In [15]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

final_orders = FastSim.state()['orders']
final_orders

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,10000.00000,10000.00000,USDC,False,1020.00000,1020.00000,1020.00000,USDC per ETH,1
1,1,ETHUSDC,ETH,10.00000,10.00000,ETH,False,2000.00000,2500.00000,2000.00000,USDC per ETH,0
2,2,ETHUSDC,USDC,10000.00000,10000.00000,USDC,False,1210.00000,1210.00000,1210.00000,USDC per ETH,3
3,3,ETHUSDC,ETH,10.00000,10.00000,ETH,False,2010.00000,2810.00000,2010.00000,USDC per ETH,2
4,4,ETHUSDC,USDC,10948.32705,10948.32705,USDC,False,1050.00000,1050.00000,1050.00000,USDC per ETH,5
5,5,ETHUSDC,ETH,10.00000,9.53154,ETH,False,2020.00000,2220.00000,2028.75457,USDC per ETH,4
6,6,ETHUSDC,USDC,30446.19557,30446.19557,USDC,False,1055.00000,1055.00000,1055.00000,USDC per ETH,7
7,7,ETHUSDC,ETH,10.00000,0.00158,ETH,False,2030.00000,2060.00000,2059.99521,USDC per ETH,6
8,8,ETHUSDC,USDC,30595.85978,30595.85978,USDC,False,1240.00000,1040.00000,1240.00000,USDC per ETH,9
9,9,ETHUSDC,ETH,10.00000,0.00152,ETH,False,2040.00000,2080.00000,2079.99381,USDC per ETH,8


# Compare the results

We can generate the comparison df using compare_results then run a check over this to see if it meets a defined tolerance

In [16]:
# pd.set_option('display.float_format', lambda x: '%.5f' % x)

# results = compare_results(strats, initial_orders, final_orders)
# results

In [17]:
# check_results(results, tolerance = 1e-1, short_long = 'short')

In [18]:
# check_results(results, tolerance = 1e-1, short_long = 'long')